In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import subprocess
from metaearth.provider.snotel import *

AttributeError: module 'shapely' has no attribute 'geometry'

In [3]:
snotel_summary = pd.read_csv('../../snotel_data/SNOTEL_summary.csv', delimiter=',', 
encoding_errors="ignore", lineterminator='\n')
snotel_summary.columns = [col.rstrip(' ') for col in snotel_summary.columns]

In [4]:
snotel_summary.columns

Index(['Num', 'SNOTEL ID', 'SNOTEL State', 'SNOTEL Name', 'SNOTEL Elev (ft)',
       'SNOTEL Lat ()', 'SNOTEL Lon ()', 'Data Starting Date',
       'Data Ending Date', 'Unnamed: 9', 'Unnamed: 10', '\r'],
      dtype='object')

In [5]:
snotel_summary.head()

Num  SNOTEL ID SNOTEL State    SNOTEL Name  SNOTEL Elev (ft)  \
0    1       1000           OR  Annie Springs              6010   
1    2       1001           AK      Long Lake               850   
2    3       1003           AK  Mcneil Canyon              1320   
3    4       1005           CO      Ute Creek             10650   
4    5       1006           NV     Lewis Peak              7400   

   SNOTEL Lat ()  SNOTEL Lon () Data Starting Date Data Ending Date  \
0          42.87        -122.17          9/15/2000        9/30/2018   
1          58.19        -133.83          10/1/1999        9/30/2018   
2          59.74        -151.25          10/1/1986        9/30/2018   
3          37.61        -105.37          8/15/2000        9/30/2018   
4          40.36        -116.86          8/23/2000        9/30/2018   

   Unnamed: 9  Unnamed: 10  \r  
0         NaN          NaN  \r  
1         NaN          NaN  \r  
2         NaN          NaN  \r  
3         NaN          NaN  \r  
4         NaN          NaN  \r

In [6]:
snotel_summary[(snotel_summary['SNOTEL Lat ()']==40.71) & (snotel_summary['SNOTEL Lon ()']==-106.01)]

Num  SNOTEL ID SNOTEL State SNOTEL Name  SNOTEL Elev (ft)  SNOTEL Lat ()  \
17   18       1032           CO       Rawah              9020          40.71   

    SNOTEL Lon () Data Starting Date Data Ending Date  Unnamed: 9  \
17        -106.01          9/12/2002        9/30/2018         NaN   

    Unnamed: 10  \r  
17          NaN  \r

In [15]:
rawah =snotel_summary[snotel_summary['SNOTEL Name']=='Rawah']
rawah

Num  SNOTEL ID SNOTEL State SNOTEL Name  SNOTEL Elev (ft)  SNOTEL Lat ()  \
17   18       1032           CO       Rawah              9020          40.71   

    SNOTEL Lon () Data Starting Date Data Ending Date  Unnamed: 9  \
17        -106.01          9/12/2002        9/30/2018         NaN   

    Unnamed: 10  \r  
17          NaN  \r

In [18]:
rawah['SNOTEL ID'].values[0]

1032

In [9]:
len(snotel_summary['SNOTEL Name'].unique())

829

In [22]:
def curl_snotel_data(id, state, network='SNTL', request_url = 'https://wcc.sc.egov.usda.gov/reportGenerator/view_csv/customSingleStationReport,metric/daily/',
    request_params = r'%7Cid=%22%22%7Cname/POR_BEGIN,POR_END/WTEQ::value,PREC::value,TMAX::value,TMIN::value,TAVG::value,PRCP::value'):
    curl_url = f'{request_url}/{id}:{state}:{network}{request_params}'
    proc = subprocess.Popen(["curl", curl_url], stdout=subprocess.PIPE)
    (out, err) = proc.communicate()
    print(err)
    return out
    

def get_snotel_data(snotel_summary=None, id=None, name=None, elevation=None,lat=None, lon=None, state=None, start_date=None, end_date=None, ):
    return_data = {}
    request_url = 'https://wcc.sc.egov.usda.gov/reportGenerator/view_csv/customSingleStationReport,metric/daily/'
    request_params = r'%7Cid=%22%22%7Cname/POR_BEGIN,POR_END/WTEQ::value,PREC::value,TMAX::value,TMIN::value,TAVG::value,PRCP::value'
    if snotel_summary is None:
        snotel_summary = pd.read_csv('../../snotel_data/SNOTEL_summary.csv', delimiter=',', 
encoding_errors="ignore", lineterminator='\n')
    snotel_summary.columns = [col.rstrip(' ') for col in snotel_summary.columns]
    if id is not None:
        if ~isinstance(id, list):
            id = [id]
        for i in id:
            curr_snotel = snotel_summary[snotel_summary['SNOTEL ID']==i]
            id = curr_snotel['SNOTEL ID'].values[0]
            state = curr_snotel['SNOTEL State'].values[0]
            return_data[i]= curl_snotel_data(id, state)

    else:
        # Return all
        pass
    
    return return_data

In [24]:
snotel_data = get_snotel_data(snotel_summary, id=1032)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 28408    0 28408    0     0  16821      0 --:--:--  0:00:01 --:--:-- 16819

None


100  245k    0  245k    0     0   104k      0 --:--:--  0:00:02 --:--:--  104k


In [88]:
id = [1, 2]
snotel_summary[snotel_summary['SNOTEL ID'].isin(id)]

Empty DataFrame
Columns: [Num, SNOTEL ID, SNOTEL State, SNOTEL Name, SNOTEL Elev (ft), SNOTEL Lat (), SNOTEL Lon (), Data Starting Date, Data Ending Date, Unnamed: 9, Unnamed: 10, 
]
Index: []